In [10]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
from copy import deepcopy

import numpy as np
import pandas as pd
from ordinor.utils.validation import check_convert_input_log

from ordinor.io import read_disco_csv
from ordinor.execution_context.rule_based import ODTMiner


In [11]:
log = 'bpic15_time_manipulated' #bpic15_amended_typed
fn = f'data/processed/{log}.csv'
df = pd.read_csv(fn)
#df.drop("", axis=1)

In [12]:
# when using bpic15_amended_typed
if(log == "bpic15_amended_typed"):
    df = df.rename(columns={
                # Resource-related
                "activityNameEN": "activity label",
                "Resource": "org:resource",
                # TT-related
                "Complete Timestamp": "time:timestamp",
                # CT-related
                'case:last_phase': 'ct:last_phase', 
                # AT-related
                "Activity": "concept:name",
                # Additional
                'case:parts': 'case_parts',
                "Case ID": "case:concept:name"
    })
df.head()

,Unnamed: 0,case:concept:name,activity label,org:resource,time:timestamp,ct:last_phase,case_parts,concept:name,r:municipality,ct:permit_type,at:phase,tt:month,tt:day,tt:weekday,tt:ampm
0,0,10009138,register submission date request,9264148,2014-04-11 06:00:00+0000,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_010,muni-1,Bouw,01_HOOFD_0,Apr,Day_10,Thu,PM
1,1,10009138,OLO messaging active,9264148,2014-04-14 06:00:00+0000,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_011,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
2,2,10009138,send confirmation receipt,9264148,2014-04-14 06:00:00+0000,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_020,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
3,5,10009138,create procedure confirmation,9264148,2014-04-14 06:00:00+0000,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_065_0,muni-1,Bouw,01_HOOFD_0,Apr,Day_13,Sun,PM
4,6,10009138,create subcases completeness,9264148,2014-04-14 06:00:00+0000,Beschikking verzonden,"Bouw,Reclame",01_HOOFD_110_0,muni-1,Bouw,01_HOOFD_1,Apr,Day_13,Sun,PM


In [4]:
#test = check_convert_input_log(df) # --> use to debug when column naming is not satisfied in the way its needed

In [13]:
# specification
spec = {
    'type_def_attrs': {
        # BPIC15
        'ct:permit_type': {'attr_type': 'categorical', 'attr_dim': 'CT'},
        'at:phase': {'attr_type': 'categorical', 'attr_dim': 'AT'},
        'tt:weekday': {'attr_type': 'categorical', 'attr_dim': 'TT'}, 
        'tt:ampm': {'attr_type': 'categorical', 'attr_dim': 'TT'},
    }
}

miner = ODTMiner(df, spec, max_height=12, trace_history=True)

C:\Users\I516663\Documents\GitHub\No-time-to-dice_group_project_IE692\OrdinoR-dev\ordinor\execution_context\rule_based\decision_tree.py:44: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self._log = el[included_cols]
C:\Users\I516663\Documents\GitHub\No-time-to-dice_group_project_IE692\OrdinoR-dev\ordinor\execution_context\rule_based\decision_tree.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._log[const.RESOURCE] = self._log[const.RESOURCE].astype('category')
C:\Users\I516663\Documents\GitHub\No-time-to-dice_group_project_IE692\OrdinoR-dev\ordinor\execution_context\rule_based\decision_tree.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

Decision tree initialized with an empty root node
	Dis. = 0.000000, Imp. = 1.000000, Harmonic Mean. = 0.000000, *** Tree has 1 leaf node(s). ***
Start to fit decision tree with max. height = 12
Tree grows by splitting all current leaf nodes on `at:phase`, step = 1 (running for 0 hours, 0 minutes and 37.7 seconds)
	Dis. = 0.000155, Imp. = 0.999941, Harmonic Mean. = 0.000310, *** Tree has 2 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `tt:weekday`, step = 2 (running for 0 hours, 1 minutes and 21.8 seconds)
	Dis. = 0.000846, Imp. = 0.999770, Harmonic Mean. = 0.001691, *** Tree has 3 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `at:phase`, step = 3 (running for 0 hours, 1 minutes and 59.6 seconds)
	Dis. = 0.003602, Imp. = 0.998144, Harmonic Mean. = 0.007179, *** Tree has 4 leaf node(s). ***
Tree grows by splitting all current leaf nodes on `at:phase`, step = 4 (running for 0 hours, 2 minutes and 26.8 seconds)
	Dis. = 0.035858, Imp. = 0.990776, 

In [6]:
def addDimensions_1(df,miner):
    def insert_only_new_atomicRule_to_list(list, atomicRule):
        for a_rule in list:
            if a_rule==atomicRule:## __eq__ method in AtomicRule class seems reasonable
                return
        else:
            list.append(atomicRule)
    def check_attr_already_exists_in_list(list,exRule):
        for i, _ in enumerate(list):
            if exRule.is_same_attr(list[i][0]): # take index 0(first rule) because all rules in that index have the same attr
                return i
        return -1
    def get_corresponding_set_as_string(value,list_of_AtomicRules):
        for atomicRule in list_of_AtomicRules:
            if value in atomicRule.attr_vals:
                
                return "-".join(atomicRule.attr_vals)
        return None
    # this method does not differentiate between Dimensions; each column with a rule will be added
    # extract all rules from leafes into one list
    all = []
    for key, node in miner._leaves.items(): 
        for i, _ in enumerate(node.at_rule.ars):
            insert_only_new_atomicRule_to_list(all,node.at_rule.ars[i])

        for i, _ in enumerate(node.ct_rule.ars):
            insert_only_new_atomicRule_to_list(all,node.ct_rule.ars[i])

        for i, _ in enumerate(node.tt_rule.ars):
            insert_only_new_atomicRule_to_list(all,node.tt_rule.ars[i])

    # create list of atomicrules indexed by their attr
    # der umweg über die list wird gegangen, da im dict nur noch der name des attr steht und daher die is_same_attr(self, other) der AtomicRule Klasse nicht mehr genutzt werden könnte --> im unwahrscheinlichen fall dass 2 spalten gleich heißen würden hier Probleme reduziert werden
    all_disaggregated = []
    for rule in all:
        # check if list has an index with rules that are the same as the current rule
        pos = check_attr_already_exists_in_list(all_disaggregated,rule)
        if pos == -1:
            l = []
            l.append(rule)
            all_disaggregated.append(l)
        else:
            all_disaggregated[pos].append(rule)

    # create dictionary out of list; key is attr, value is list of attr_vals(which is a frozen set)
    all_disaggregated_dict = {}

    for i, _ in enumerate(all_disaggregated):
        all_disaggregated_dict[all_disaggregated[i][0].attr] = all_disaggregated[i]#any item in all_disaggregated[i] works; [0] is always available

    df = pd.read_csv("./data/processed/bpic15.csv")
    prefix="RES_"

    # initialize all result columns with None --> coulumns were no rules were learned on will still be None
    for attr_spec,_ in spec["type_def_attrs"].items():
        df[prefix+attr_spec] = None
        

    # add new columns to df that represent the learned rules
    for key, list_of_sets in all_disaggregated_dict.items():
        if key: #check if key is None
            if not key in df.columns: 
                raise Exception(f"{key=} not found in dataframe")
            df[prefix+str(key)] = df[key].apply(lambda x: get_corresponding_set_as_string(x,list_of_sets))

    # combine columns to CO columnn
    for col in df.columns:
        if col.startswith(prefix):
            if prefix+"CO" not in df.columns:
                df[prefix+"CO"] = df[col].astype(str)
            else:   
                df[prefix+"CO"] = df[prefix+"CO"] + " | " + df[col].astype(str)

    return df

In [14]:
def addDimensions_2(df,miner):  
    # this method is recommended over 1 and 3
    
    # get different rules
    l_rules_ct, l_rules_at, l_rules_tt = miner._parse_rules_from_leaves(miner._leaves)

    # label ATs
    dfs = [r.apply(df, index_only=False).assign(AT_rule=r) for r in l_rules_at]
    df = pd.concat(dfs)

    # label CTs
    dfs = [r.apply(df, index_only=False).assign(CT_rule=r) for r in l_rules_ct]
    df = pd.concat(dfs)

    # label TTs
    dfs = [r.apply(df, index_only=False).assign(TT_rule=r) for r in l_rules_tt]
    df = pd.concat(dfs)

    # add CO_rule as combination of AT CT and TT
    from ordinor.execution_context.rule_based.Rule import Rule
    df['CO_rule'] = df.apply(lambda x: Rule(x['AT_rule'].ars+x['CT_rule'].ars+x['TT_rule'].ars), axis=1)
    
    return df

In [8]:
def addDimensions_3(df,miner):
    # this method only works on the df where the miner was trained on (works by joining on index)
    df = pd.read_csv("./data/processed/bpic15.csv")
    dfs_temp = []

    for _, n in miner._leaves.items():
        df_temp_temp = pd.DataFrame({"ind": n.event_ids, "Node": [n]*len(n.event_ids)})
        dfs_temp.append(df_temp_temp)

    df_temp = pd.concat(dfs_temp)
    df = df.join(df_temp.set_index("ind"))
    df["CO_rule"] = df.apply(lambda x: x["Node"].composite_rule if not isinstance(x["Node"],float) else None,axis=1)
    df[["CT_rule", "AT_rule", "TT_rule"]] = df.apply(lambda x: pd.Series(x["Node"].composite_rule.to_types()) if not isinstance(x["Node"],float) else (None,None,None), axis=1)

    return df

In [15]:
df = pd.read_csv(f"./data/processed/{log}.csv")
df = df.rename(columns={
            # Resource-related
            'Case ID' : 'case:concept:name',
            'Complete Timestamp': 'time:timestamp',
            "Resource": "org:resource",
            "action_code": "concept:name",
        })
#df = df[df['case:concept:name'] != 4020737]
df = addDimensions_2(df,miner)
df.to_csv('output.csv', index=False)
df.head()



,Unnamed: 0,case:concept:name,activity label,org:resource,time:timestamp,ct:last_phase,case_parts,concept:name,r:municipality,ct:permit_type,at:phase,tt:month,tt:day,tt:weekday,tt:ampm,AT_rule,CT_rule,TT_rule,CO_rule
13044,16631,3462821,enter senddate continuation,560589,2011-06-22 06:00:00+0000,Procedure afgebroken,Bouw,01_HOOFD_600_2,muni-1,Bouw,01_HOOFD_6,Jun,Day_21,Tue,PM,(`at:phase` ELEMENT OF {'01_HOOFD_6'}),T(NULL),"(`tt:weekday` ELEMENT OF {'Mon', 'Sun', 'Thu',...",(T(NULL)) AND (`at:phase` ELEMENT OF {'01_HOOF...
13055,16648,3462821,send continuation letter,560589,2011-06-22 17:52:04+0000,Procedure afgebroken,Bouw,01_HOOFD_600_1,muni-1,Bouw,01_HOOFD_6,Jun,Day_22,Wed,AM,(`at:phase` ELEMENT OF {'01_HOOFD_6'}),T(NULL),"(`tt:weekday` ELEMENT OF {'Mon', 'Sun', 'Thu',...",(T(NULL)) AND (`at:phase` ELEMENT OF {'01_HOOF...
60981,81947,3657921,enter senddate continuation,560458,2011-05-03 06:00:00+0000,Zaak afgehandeld,Bouw,01_HOOFD_600_2,muni-2,Bouw,01_HOOFD_6,May,Day_2,Mon,PM,(`at:phase` ELEMENT OF {'01_HOOFD_6'}),T(NULL),"(`tt:weekday` ELEMENT OF {'Mon', 'Sun', 'Thu',...",(T(NULL)) AND (`at:phase` ELEMENT OF {'01_HOOF...
60984,81950,3657921,send continuation letter,560458,2011-05-03 23:34:09+0000,Zaak afgehandeld,Bouw,01_HOOFD_600_1,muni-2,Bouw,01_HOOFD_6,May,Day_3,Tue,PM,(`at:phase` ELEMENT OF {'01_HOOFD_6'}),T(NULL),"(`tt:weekday` ELEMENT OF {'Mon', 'Sun', 'Thu',...",(T(NULL)) AND (`at:phase` ELEMENT OF {'01_HOOF...
73691,99138,3197901,enter senddate continuation,2013365,2011-03-16 08:00:00+0000,Advies bekend,Bouw,01_HOOFD_600_2,muni-3,Bouw,01_HOOFD_6,Mar,Day_15,Tue,PM,(`at:phase` ELEMENT OF {'01_HOOFD_6'}),T(NULL),"(`tt:weekday` ELEMENT OF {'Mon', 'Sun', 'Thu',...",(T(NULL)) AND (`at:phase` ELEMENT OF {'01_HOOF...
